In [1]:
import pandas as pd

In [2]:
file = 'Resources/walkability.csv'
df = pd.read_csv(file, encoding="ISO-8859-1", low_memory=False)

In [4]:
#ADD LEADING ZEROS IN FRONT OF STATE AND COUNTY TO MATCH WITH THE SCHOOL DATA 'CNTY' KEY
df['STATEFP'] = df['STATEFP'].astype(str).apply('{:0>2}'.format)
df['COUNTYFP'] = df['COUNTYFP'].astype(str).apply('{:0>3}'.format)

#CONCATENATING 'STATEFP' WITH 'COUNTYFP' AND CREATING 'CNTY' COLUMN AS A PRIMARY KEY
df['CNTY'] = df['STATEFP'] + df['COUNTYFP']

In [5]:
#Break apart this large dataset into dataframes that will could be used to populate map visuals.  

administrative_df = df[['STATEFP','COUNTYFP', 'CBSA_Name']]
population_df = df[['STATEFP','COUNTYFP', 'TotPop', 'HH']]
autoOwnership_df = df[['STATEFP','COUNTYFP', 'AutoOwn0', 'Pct_AO0', 'AutoOwn1', 'Pct_AO1', 'AutoOwn2p', 'Pct_AO2p']]
householdWages_df = df[['STATEFP','COUNTYFP', 'Workers', 'R_LowWageWk', 'R_MedWageWk', 'R_HiWageWk', 'R_PCTLOWWAGE']]
workDistance_df = df[['STATEFP','COUNTYFP', 'D5AR', 'D5AE']]
walkability_df = df[['STATEFP','COUNTYFP','NatWalkInd']]

In [6]:
#rename the columns to better understand the data. See Resources/documentation_guide.pdf for more context. 

# ----- administrative_df ----- # 
    #area_name: Census Bureau Statistical Area to be used as a reference only. 

# ----- population_df ----- # 
    #population: is the population of each area. The total sum represents on total country population (pre 2022 census)
    #households: How many homes/units in each area. 

# ----- autoOwnership_df ----- # 
    # _cars: Each column represents how many cars per household and what percentage of total cars per area. 

# ----- householdWages_df ----- # 
    # _wage: Each column represents the wages per area.
        #low_wage: $1250/month or less 
        #medium_wage: more than $1250/month but less than $3333/month
        #high_wage: earning $3333/month or more
    #pct_low_wage: percentage of low wage workers in the area.     

# ----- workDistance_df ----- # 
    #jobs_within_45min_drive: available jobs within 45 min drive from each area. 
    #workers_within_45min_drive: how many working age people living 45 minutes from available jobs.

# ----- walkability_df ----- # 
#walkability_score:
    #walkability score = (w/3) + (x/3) + (y/6) + (z/6)
#        Where w = CBG ranked score for intersection density
#                    x = CBG ranked score for proximity to transit stops
#                    y = CBG ranked score for employment mix
#                    z = CBG ranked score for employment and household mix 


administrative_df = administrative_df.rename(columns={'STATEFP':'state','COUNTYFP':'county', 'CBSA_Name':'area_name'})
population_df = population_df.rename(columns={'STATEFP':'state','COUNTYFP':'county', 'TotPop':'population', 'HH':'households'})
autoOwnership_df = autoOwnership_df.rename(columns={'STATEFP':'state','COUNTYFP':'county', 'AutoOwn0':'no_cars', 'Pct_AO0':'pct_no_cars', 'AutoOwn1':'one_cars', 'Pct_AO1':'pct_one_cars', 'AutoOwn2p':'two+_cars', 'Pct_AO2p':'pct_two+_cars'})
householdWages_df = householdWages_df.rename(columns={'STATEFP':'state','COUNTYFP':'county','R_LowWageWk':'low_wage', 'R_MedWageWk':'medium_wage', 'R_HiWageWk':'high_wage', 'R_PCTLOWWAGE':'pct_low_wage'})
workDistance_df = workDistance_df.rename(columns={'STATEFP':'state','COUNTYFP':'county','D5AR':'jobs_within_45min_drive', 'D5AE': 'workers_within_45min_drive'})
walkability_df = walkability_df.rename(columns={'STATEFP':'state','COUNTYFP':'county','NatWalkInd':'walkability_score'})

In [7]:
#Grouping data from area into county that will be used for county data analysis and visualizations. Rounding and 
#Reseting the index to allow combining the state and county code in the next step.

county_population_df = population_df.groupby(['state','county']).sum().round().reset_index()
county_autoOwnership_df = autoOwnership_df.groupby(['state','county']).sum().round().reset_index()
county_householdWages_df = householdWages_df.groupby(['state','county']).mean().round().reset_index()
county_workDistance_df = workDistance_df.groupby(['state','county']).mean().round().reset_index()
county_walkability_df = walkability_df.groupby(['state','county']).mean().reset_index()

In [8]:
#Adding a unique_id that combines the state and county code. This could be used as a unique key because county codes are not unique. 

administrative_df['unique_id'] = administrative_df['state']+administrative_df['county']
county_population_df['unique_id'] = county_population_df['state']+county_population_df['county']
county_autoOwnership_df['unique_id'] = county_autoOwnership_df['state']+county_autoOwnership_df['county']
county_householdWages_df['unique_id'] = county_householdWages_df['state']+county_householdWages_df['county']
county_workDistance_df['unique_id'] = county_workDistance_df['state']+county_workDistance_df['county']
county_walkability_df['unique_id'] = county_walkability_df['state']+county_walkability_df['county']

In [9]:
#Exporting dataframes to CSV files. This is needed because the Walkability.csv file is too large on it's own to upload onto Github. 

administrative_df.to_csv('Resources/administrative_df.csv')
county_population_df.to_csv('Resources/county_population_df.csv')
county_autoOwnership_df.to_csv('Resources/county_autoOwnership_df.csv')
county_householdWages_df.to_csv('Resources/county_householdWages_df.csv')
county_workDistance_df.to_csv('Resources/county_workDistance_df.csv')
county_walkability_df.to_csv('Resources/county_walkability_df.csv')